# Pedestiran Counts Virtual Sensor

In [991]:
import pandas as pd
import geopy
from geopy import distance

In [1733]:
#df = pd.read_csv('pedestrian_feb.csv')
df2 = pd.read_csv('Pedestrian_Counting_System_-_Sensor_Locations.csv')

In [1682]:
df = pd.read_csv("pedestrian_temp.csv")

In [1683]:
df.head()

date_time  sensor_id     sensor_name  hourly_counts   latitude  \
0  01/10/2018 0:00          5  Princes Bridge            123 -37.818742   
1  01/10/2018 1:00          5  Princes Bridge             46 -37.818742   
2  01/10/2018 2:00          5  Princes Bridge             25 -37.818742   
3  01/10/2018 3:00          5  Princes Bridge             48 -37.818742   
4  01/10/2018 4:00          5  Princes Bridge             50 -37.818742   

    longitude  
0  144.967877  
1  144.967877  
2  144.967877  
3  144.967877  
4  144.967877

## Distance Functions

In [1684]:
def within(df, target, radius, g):
    
    temp = []
    
    for x in range(len(target)):
        # Use try-except for error handling
        try:
            # Find coordinate1,2 calculate distance in km
            coords_1 = (df["Latitude"][g], df["Longitude"][g])
            coords_2 = (target["latitude"][x], target["longitude"][x])
            dist = geopy.distance.distance(coords_1, coords_2).km

            if dist <= radius:
                temp.append(target.iloc[x])
        except:
            pass
    return temp

In [1685]:
def within_d(df, target, radius, g):
    
    distances = []
    
    for x in range(len(target)):
        # Use try-except for error handling
        try:
            # Find coordinate1,2 calculate distance in km
            coords_1 = (df["Latitude"][g], df["Longitude"][g])
            coords_2 = (target["latitude"][x], target["longitude"][x])
            dist = geopy.distance.distance(coords_1, coords_2).km

            if dist <= radius:
                distances.append(dist)
        except:
            pass
    return distances

## Assign Lat, Lon for each sensor

In [1734]:
df.head(2)

date_time  sensor_id     sensor_name  hourly_counts   latitude  \
0  01/10/2018 0:00          5  Princes Bridge            123 -37.818742   
1  01/10/2018 1:00          5  Princes Bridge             46 -37.818742   

    longitude  
0  144.967877  
1  144.967877

In [1735]:
df2.head(1)

sensor_id          sensor_description sensor_name installation_date status  \
0          1  Bourke Street Mall (North)    Bou292_T        24/03/2009      A   

   note direction_1 direction_2   latitude   longitude  \
0   NaN        West        East -37.813494  144.965153   

                      location  
0  (-37.8134944, 144.96515324)

In [1736]:
df2 = df2.iloc[:,[0,8,9]]

In [1737]:
df2.tail(2)

sensor_id   latitude   longitude
49         58 -37.816861  144.953581
50         59 -37.808256  144.963049

In [1738]:
# Merge on sensor id to get lat, long for each location
df3 = pd.merge(df, df2, on='sensor_id', how='outer').iloc[:,[0,1,2,3,6,7]]
df3.rename(columns={'latitude_y':'latitude', 'longitude_y':'longitude'}, inplace=True)

In [1739]:
df3.tail()

date_time  sensor_id  sensor_name  hourly_counts   latitude  \
214029  31/03/2019 21:00          8  Webb Bridge           51.0 -37.822935   
214030  31/03/2019 22:00          8  Webb Bridge           27.0 -37.822935   
214031  31/03/2019 23:00          8  Webb Bridge           27.0 -37.822935   
214032               NaN          3          NaN            NaN -37.811015   
214033               NaN         59          NaN            NaN -37.808256   

         longitude  
214029  144.947175  
214030  144.947175  
214031  144.947175  
214032  144.964295  
214033  144.963049

In [1740]:
df3 = df3.drop(df3.index[[214032,214033]])

In [1741]:
sensor_list = df3['sensor_id'].unique()

In [1742]:
df3=df3.drop_duplicates(keep='first')

In [1743]:
for sensor in sensor_list:
    if len(df3[df3['sensor_id']==sensor])!=4368:
        print(sensor)
    else:
        pass

In [1744]:
df3.tail()

date_time  sensor_id  sensor_name  hourly_counts   latitude  \
214027  31/03/2019 19:00          8  Webb Bridge          134.0 -37.822935   
214028  31/03/2019 20:00          8  Webb Bridge           85.0 -37.822935   
214029  31/03/2019 21:00          8  Webb Bridge           51.0 -37.822935   
214030  31/03/2019 22:00          8  Webb Bridge           27.0 -37.822935   
214031  31/03/2019 23:00          8  Webb Bridge           27.0 -37.822935   

         longitude  
214027  144.947175  
214028  144.947175  
214029  144.947175  
214030  144.947175  
214031  144.947175

In [1724]:
df3[df3['sensor_id']==110]

Empty DataFrame
Columns: [date_time, sensor_id, sensor_name, hourly_counts, latitude, longitude]
Index: []

In [1213]:
#df3_new.to_csv('../data/ped_working2.csv', index=False)
df3 = pd.read_csv("../data/ped_working2.csv")

In [374]:
df3['date_time'] = pd.to_datetime(df3['date_time'])

## Create Virtual Sensor

### <font color='blue'>Sensor A - Collins St</font>


In [1730]:
sensor1 = pd.DataFrame([{'Latitude':-37.817771,'Longitude':144.957951}])
sensor1

Latitude   Longitude
0 -37.817771  144.957951

### Nearest sensors

In [1745]:
# Find Sensor information and Distance
df4 = pd.concat([pd.DataFrame(within(sensor1, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor1, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1746]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').head(3)
df4 = df4.reset_index(drop=True)
df4.head(3)

sensor_id   latitude   longitude  distance
0       36.0 -37.816525  144.961211  0.318577
1       24.0 -37.818880  144.954492  0.328479
2       23.0 -37.819093  144.954527  0.335229

In [1773]:
a,b,c = 0,1,2
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1777]:
sensor_a = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [1778]:
sensor_a['latitude']= sensor1['Latitude'][0]
sensor_a['longitude']= sensor1['Longitude'][0]
sensor_a['sensor_name']= 'Collin St - Virtual'
sensor_a['sensor_id']= 101
sensor_a.tail(10)

date_time  hourly_counts   latitude   longitude  \
4358  31/03/2019 14:00          563.0 -37.817771  144.957951   
4359  31/03/2019 15:00          552.0 -37.817771  144.957951   
4360  31/03/2019 16:00          666.0 -37.817771  144.957951   
4361  31/03/2019 17:00          523.0 -37.817771  144.957951   
4362  31/03/2019 18:00          474.0 -37.817771  144.957951   
4363  31/03/2019 19:00          376.0 -37.817771  144.957951   
4364  31/03/2019 20:00          314.0 -37.817771  144.957951   
4365  31/03/2019 21:00          240.0 -37.817771  144.957951   
4366  31/03/2019 22:00          189.0 -37.817771  144.957951   
4367  31/03/2019 23:00          104.0 -37.817771  144.957951   

              sensor_name  sensor_id  
4358  Collin St - Virtual        101  
4359  Collin St - Virtual        101  
4360  Collin St - Virtual        101  
4361  Collin St - Virtual        101  
4362  Collin St - Virtual        101  
4363  Collin St - Virtual        101  
4364  Collin St - Virtual        101  
4365  Collin St - Virtual        101  
4366  Collin St - Virtual        101  
4367  Collin St - Virtual        101

### Append Existing DF

In [1779]:
df3_new = df3.append(sensor_a, sort=False).reset_index(drop=True)
df2 = df2.append(sensor_a.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

In [1780]:
df3_new.tail(10)

date_time  sensor_id          sensor_name  hourly_counts  \
218390  31/03/2019 14:00        101  Collin St - Virtual          563.0   
218391  31/03/2019 15:00        101  Collin St - Virtual          552.0   
218392  31/03/2019 16:00        101  Collin St - Virtual          666.0   
218393  31/03/2019 17:00        101  Collin St - Virtual          523.0   
218394  31/03/2019 18:00        101  Collin St - Virtual          474.0   
218395  31/03/2019 19:00        101  Collin St - Virtual          376.0   
218396  31/03/2019 20:00        101  Collin St - Virtual          314.0   
218397  31/03/2019 21:00        101  Collin St - Virtual          240.0   
218398  31/03/2019 22:00        101  Collin St - Virtual          189.0   
218399  31/03/2019 23:00        101  Collin St - Virtual          104.0   

         latitude   longitude  
218390 -37.817771  144.957951  
218391 -37.817771  144.957951  
218392 -37.817771  144.957951  
218393 -37.817771  144.957951  
218394 -37.817771  144.957951  
218395 -37.817771  144.957951  
218396 -37.817771  144.957951  
218397 -37.817771  144.957951  
218398 -37.817771  144.957951  
218399 -37.817771  144.957951

### <font color='blue'>Sensor B - Bourke St</font>

In [1781]:
sensor2 = pd.DataFrame([{'Latitude':-37.81447,'Longitude':144.96197}])
sensor2

Latitude  Longitude
0 -37.81447  144.96197

### Nearest sensors

In [1782]:
# Find Sensor information and Distance
df4 = pd.concat([pd.DataFrame(within(sensor2, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor2, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1783]:
# Nearest 3 sensors
df4 = df4.sort_values('distance')
df4 = df4.reset_index(drop=True)
df4.head()

sensor_id   latitude   longitude  distance
0       52.0 -37.812522  144.961940  0.216278
1       36.0 -37.816525  144.961211  0.237719
2       56.0 -37.812348  144.961533  0.238676
3        2.0 -37.813807  144.965167  0.290993
4        1.0 -37.813494  144.965153  0.300483

In [1784]:
a,b,c = 0,1,2
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1785]:
sensor_b = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [1786]:
sensor_b.head(1)

date_time  hourly_counts
0  01/10/2018 0:00           40.0

In [1787]:
sensor_b['latitude']= sensor2['Latitude'][0]
sensor_b['longitude']= sensor2['Longitude'][0]
sensor_b['sensor_name']= 'Bourke St - Virtual'
sensor_b['sensor_id']= 102
sensor_b.head(10)

date_time  hourly_counts  latitude  longitude          sensor_name  \
0  01/10/2018 0:00           40.0 -37.81447  144.96197  Bourke St - Virtual   
1  01/10/2018 1:00           16.0 -37.81447  144.96197  Bourke St - Virtual   
2  01/10/2018 2:00           18.0 -37.81447  144.96197  Bourke St - Virtual   
3  01/10/2018 3:00            7.0 -37.81447  144.96197  Bourke St - Virtual   
4  01/10/2018 4:00            7.0 -37.81447  144.96197  Bourke St - Virtual   
5  01/10/2018 5:00           13.0 -37.81447  144.96197  Bourke St - Virtual   
6  01/10/2018 6:00           45.0 -37.81447  144.96197  Bourke St - Virtual   
7  01/10/2018 7:00          202.0 -37.81447  144.96197  Bourke St - Virtual   
8  01/10/2018 8:00          608.0 -37.81447  144.96197  Bourke St - Virtual   
9  01/10/2018 9:00          513.0 -37.81447  144.96197  Bourke St - Virtual   

   sensor_id  
0        102  
1        102  
2        102  
3        102  
4        102  
5        102  
6        102  
7        102  
8        102  
9        102

### Append Existing DF

In [1788]:
df3_new = df3_new.append(sensor_b, sort=False).reset_index(drop=True)

In [1789]:
df2 = df2.append(sensor_b.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

In [759]:
#df3_new['date_time'] = pd.to_datetime(df3_new['date_time']).dt.strftime('%d/%m/%Y %H:%M')

### <font color='blue'>Sensor C - sensor 13</font>

In [1790]:
sensor3 = pd.DataFrame([{'Latitude':-37.812397,'Longitude':144.956526}])
sensor3

Latitude   Longitude
0 -37.812397  144.956526

### Nearest sensors

In [1791]:
# Find Sensor information and Distance
df4 = pd.concat([pd.DataFrame(within(sensor3, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor3, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1792]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[1:5,:]
df4 = df4.reset_index(drop=True)
df4.head(3)

sensor_id   latitude   longitude  distance
0       52.0 -37.812522  144.961940  0.476943
1       51.0 -37.808418  144.959063  0.494924
2       47.0 -37.811603  144.962201  0.507407

In [1793]:
a,b,c = 0,1,2
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1794]:
sensor_c = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [1795]:
sensor_c.head(1)

date_time  hourly_counts
0  01/10/2018 0:00           64.0

In [1796]:
sensor_c['latitude']= sensor3['Latitude'][0]
sensor_c['longitude']= sensor3['Longitude'][0]
sensor_c['sensor_name']= 'Sensor13 - Virtual'
sensor_c['sensor_id']= 13
sensor_c.tail()

date_time  hourly_counts   latitude   longitude  \
4363  31/03/2019 19:00          580.0 -37.812397  144.956526   
4364  31/03/2019 20:00          417.0 -37.812397  144.956526   
4365  31/03/2019 21:00          311.0 -37.812397  144.956526   
4366  31/03/2019 22:00          219.0 -37.812397  144.956526   
4367  31/03/2019 23:00          129.0 -37.812397  144.956526   

             sensor_name  sensor_id  
4363  Sensor13 - Virtual         13  
4364  Sensor13 - Virtual         13  
4365  Sensor13 - Virtual         13  
4366  Sensor13 - Virtual         13  
4367  Sensor13 - Virtual         13

### Append Existing DF

In [1797]:
df3_new = df3_new.append(sensor_c, sort=False).reset_index(drop=True)

In [1798]:
df2 = df2.append(sensor_c.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor D - Queen St</font>

In [1799]:
sensor4 = pd.DataFrame([{'Latitude':-37.810979,'Longitude':144.958897}])
sensor4

Latitude   Longitude
0 -37.810979  144.958897

### Nearest sensors

In [1800]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor4, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor4, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1801]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[[0,2,3],:]
df4 = df4.reset_index(drop=True)
df4.head()

sensor_id   latitude   longitude  distance
0       13.0 -37.812397  144.956526  0.261459
1       51.0 -37.808418  144.959063  0.284614
2       26.0 -37.808695  144.960494  0.289861

In [1802]:
a,b,c = 0,1,2
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1803]:
## Change sensor

sensor_d = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [1804]:
sensor_d.head(5)

date_time  hourly_counts
0  01/10/2018 0:00           77.0
1  01/10/2018 1:00           39.0
2  01/10/2018 2:00           26.0
3  01/10/2018 3:00           21.0
4  01/10/2018 4:00           12.0

In [1805]:
## Change sensor

sensor_d['latitude']= sensor4['Latitude'][0]
sensor_d['longitude']= sensor4['Longitude'][0]
sensor_d['sensor_name']= 'Queen ST - Virtual'
sensor_d['sensor_id']= 103
sensor_d.head()

date_time  hourly_counts   latitude   longitude         sensor_name  \
0  01/10/2018 0:00           77.0 -37.810979  144.958897  Queen ST - Virtual   
1  01/10/2018 1:00           39.0 -37.810979  144.958897  Queen ST - Virtual   
2  01/10/2018 2:00           26.0 -37.810979  144.958897  Queen ST - Virtual   
3  01/10/2018 3:00           21.0 -37.810979  144.958897  Queen ST - Virtual   
4  01/10/2018 4:00           12.0 -37.810979  144.958897  Queen ST - Virtual   

   sensor_id  
0        103  
1        103  
2        103  
3        103  
4        103

### Append Existing DF

In [1806]:
## Change sensor

df3_new = df3_new.append(sensor_d, sort=False).reset_index(drop=True)

In [1807]:
## Change sensor

df2 = df2.append(sensor_d.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor E - Hyatt</font>

In [1808]:
sensor5 = pd.DataFrame([{'Latitude':-37.81550,'Longitude':144.97005}])
sensor5

Latitude  Longitude
0  -37.8155  144.97005

### Nearest sensors

In [1809]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor5, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor5, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1810]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(3)

sensor_id   latitude   longitude  distance
0       39.0 -37.813797  144.969957  0.189142
1       18.0 -37.813449  144.973053  0.348979
2       17.0 -37.813625  144.973236  0.349267

In [1811]:
a,b,c = 0,1,2
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1812]:
## Change sensor

sensor_e = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [1813]:
sensor_e.head(5)

date_time  hourly_counts
0  01/10/2018 0:00            8.0
1  01/10/2018 1:00            4.0
2  01/10/2018 2:00            3.0
3  01/10/2018 3:00            6.0
4  01/10/2018 4:00           27.0

In [1814]:
## Change sensor

sensor_e['latitude']= sensor5['Latitude'][0]
sensor_e['longitude']= sensor5['Longitude'][0]
sensor_e['sensor_name']= 'Hyatt - Virtual'
sensor_e['sensor_id']= 104
sensor_e.head()

date_time  hourly_counts  latitude  longitude      sensor_name  \
0  01/10/2018 0:00            8.0  -37.8155  144.97005  Hyatt - Virtual   
1  01/10/2018 1:00            4.0  -37.8155  144.97005  Hyatt - Virtual   
2  01/10/2018 2:00            3.0  -37.8155  144.97005  Hyatt - Virtual   
3  01/10/2018 3:00            6.0  -37.8155  144.97005  Hyatt - Virtual   
4  01/10/2018 4:00           27.0  -37.8155  144.97005  Hyatt - Virtual   

   sensor_id  
0        104  
1        104  
2        104  
3        104  
4        104

In [1815]:
sensor_e.tail()

date_time  hourly_counts  latitude  longitude      sensor_name  \
4363  31/03/2019 19:00          140.0  -37.8155  144.97005  Hyatt - Virtual   
4364  31/03/2019 20:00          111.0  -37.8155  144.97005  Hyatt - Virtual   
4365  31/03/2019 21:00           91.0  -37.8155  144.97005  Hyatt - Virtual   
4366  31/03/2019 22:00           64.0  -37.8155  144.97005  Hyatt - Virtual   
4367  31/03/2019 23:00           29.0  -37.8155  144.97005  Hyatt - Virtual   

      sensor_id  
4363        104  
4364        104  
4365        104  
4366        104  
4367        104

### Append Existing DF

In [1816]:
## Change sensor
df3_new = df3_new.append(sensor_e, sort=False).reset_index(drop=True)

In [1817]:
## Change sensor

df2 = df2.append(sensor_e.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor F - Exhibition St</font>

In [1818]:
sensor6 = pd.DataFrame([{'Latitude':-37.808234,'Longitude':144.968757}])
sensor6

Latitude   Longitude
0 -37.808234  144.968757

### Nearest sensors

In [1819]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor6, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor6, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1820]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(3)

sensor_id   latitude   longitude  distance
0       40.0 -37.809993  144.972276  0.366274
1       30.0 -37.811218  144.966568  0.383255
2       20.0 -37.811729  144.968247  0.390529

In [1821]:
a,b,c = 0,1,2
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1822]:
## Change sensor

sensor_f = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [1823]:
sensor_f.head(5)

date_time  hourly_counts
0  01/10/2018 0:00           84.0
1  01/10/2018 1:00           54.0
2  01/10/2018 2:00           47.0
3  01/10/2018 3:00           32.0
4  01/10/2018 4:00           38.0

In [1824]:
## Change sensor

sensor_f['latitude']= sensor6['Latitude'][0]
sensor_f['longitude']= sensor6['Longitude'][0]
sensor_f['sensor_name']= 'Exhibition St - Virtual'
sensor_f['sensor_id']= 105
sensor_f.head()

date_time  hourly_counts   latitude   longitude  \
0  01/10/2018 0:00           84.0 -37.808234  144.968757   
1  01/10/2018 1:00           54.0 -37.808234  144.968757   
2  01/10/2018 2:00           47.0 -37.808234  144.968757   
3  01/10/2018 3:00           32.0 -37.808234  144.968757   
4  01/10/2018 4:00           38.0 -37.808234  144.968757   

               sensor_name  sensor_id  
0  Exhibition St - Virtual        105  
1  Exhibition St - Virtual        105  
2  Exhibition St - Virtual        105  
3  Exhibition St - Virtual        105  
4  Exhibition St - Virtual        105

### Append Existing DF

In [1825]:
## Change sensor

df3_new = df3_new.append(sensor_f, sort=False).reset_index(drop=True)

In [1826]:
## Change sensor

df2 = df2.append(sensor_f.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor G - La Trobe St</font>

In [1827]:
sensor7 = pd.DataFrame([{'Latitude':-37.81319,'Longitude':144.95174}])
sensor7

Latitude  Longitude
0 -37.81319  144.95174

### Nearest sensors

In [1828]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor7, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor7, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1829]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(3)

sensor_id   latitude   longitude  distance
0       13.0 -37.812397  144.956526  0.430526
1       58.0 -37.816861  144.953581  0.438484
2       57.0 -37.817673  144.950256  0.514507

In [1830]:
a,b,c = 0,1,2
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1831]:
## Change sensor

sensor_g = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [1832]:
sensor_g.head(5)

date_time  hourly_counts
0  01/10/2018 0:00           60.0
1  01/10/2018 1:00           25.0
2  01/10/2018 2:00           18.0
3  01/10/2018 3:00           11.0
4  01/10/2018 4:00           16.0

In [1833]:
## Change sensor

sensor_g['latitude']= sensor7['Latitude'][0]
sensor_g['longitude']= sensor7['Longitude'][0]
sensor_g['sensor_name']= 'La Trobe St - Virtual'
sensor_g['sensor_id']= 106
sensor_g.head()

date_time  hourly_counts  latitude  longitude            sensor_name  \
0  01/10/2018 0:00           60.0 -37.81319  144.95174  La Trobe St - Virtual   
1  01/10/2018 1:00           25.0 -37.81319  144.95174  La Trobe St - Virtual   
2  01/10/2018 2:00           18.0 -37.81319  144.95174  La Trobe St - Virtual   
3  01/10/2018 3:00           11.0 -37.81319  144.95174  La Trobe St - Virtual   
4  01/10/2018 4:00           16.0 -37.81319  144.95174  La Trobe St - Virtual   

   sensor_id  
0        106  
1        106  
2        106  
3        106  
4        106

### Append Existing DF

In [1834]:
## Change sensor

df3_new = df3_new.append(sensor_g, sort=False).reset_index(drop=True)

In [1835]:
## Change sensor

df2 = df2.append(sensor_g.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor H - William St</font>

In [1836]:
sensor8 = pd.DataFrame([{'Latitude':-37.81563,'Longitude':144.95823}])
sensor8

Latitude  Longitude
0 -37.81563  144.95823

### Nearest sensors

In [1837]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor8, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor8, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1838]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(5)

sensor_id   latitude   longitude  distance
0      101.0 -37.817771  144.957951  0.238902
1       36.0 -37.816525  144.961211  0.280628
2      102.0 -37.814470  144.961970  0.353591
3       13.0 -37.812397  144.956526  0.388947

In [1839]:
a,b,c = 0,1,2
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1840]:
## Change sensor

sensor_h = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

In [1841]:
sensor_h.head(5)

date_time  hourly_counts
0  01/10/2018 0:00           39.0
1  01/10/2018 1:00           13.0
2  01/10/2018 2:00           13.0
3  01/10/2018 3:00            7.0
4  01/10/2018 4:00           13.0

In [1842]:
## Change sensor

sensor_h['latitude']= sensor8['Latitude'][0]
sensor_h['longitude']= sensor8['Longitude'][0]
sensor_h['sensor_name']= 'Willam St - Virtual'
sensor_h['sensor_id']= 107
sensor_h.head()

date_time  hourly_counts  latitude  longitude          sensor_name  \
0  01/10/2018 0:00           39.0 -37.81563  144.95823  Willam St - Virtual   
1  01/10/2018 1:00           13.0 -37.81563  144.95823  Willam St - Virtual   
2  01/10/2018 2:00           13.0 -37.81563  144.95823  Willam St - Virtual   
3  01/10/2018 3:00            7.0 -37.81563  144.95823  Willam St - Virtual   
4  01/10/2018 4:00           13.0 -37.81563  144.95823  Willam St - Virtual   

   sensor_id  
0        107  
1        107  
2        107  
3        107  
4        107

### Append Existing DF

In [1843]:
## Change sensor

df3_new = df3_new.append(sensor_h, sort=False).reset_index(drop=True)

In [1844]:
## Change sensor

df2 = df2.append(sensor_h.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor I - Sea Life </font>

In [1845]:
sensor9 = pd.DataFrame([{'Latitude':-37.82038,'Longitude':144.95746}])
sensor9

Latitude  Longitude
0 -37.82038  144.95746

### Nearest sensors

In [1846]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor9, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor9, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1847]:
# Nearest 3 sensors
df4 = df4.sort_values('distance').iloc[0:4,:]
df4 = df4.reset_index(drop=True)
df4.head(5)

sensor_id   latitude   longitude  distance
0      101.0 -37.817771  144.957951  0.292790
1       23.0 -37.819093  144.954527  0.295103
2       24.0 -37.818880  144.954492  0.309876
3       25.0 -37.824018  144.956044  0.422568

In [1848]:
a,b,c = 0,2,3
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1849]:
## Change sensor

sensor_i = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

sensor_i.head(5)

date_time  hourly_counts
0  01/10/2018 0:00           76.0
1  01/10/2018 1:00           37.0
2  01/10/2018 2:00           18.0
3  01/10/2018 3:00            9.0
4  01/10/2018 4:00           36.0

In [1850]:
## Change sensor

sensor_i['latitude']= sensor9['Latitude'][0]
sensor_i['longitude']= sensor9['Longitude'][0]
sensor_i['sensor_name']= 'Sea Life - Virtual'
sensor_i['sensor_id']= 108
sensor_i.head()

date_time  hourly_counts  latitude  longitude         sensor_name  \
0  01/10/2018 0:00           76.0 -37.82038  144.95746  Sea Life - Virtual   
1  01/10/2018 1:00           37.0 -37.82038  144.95746  Sea Life - Virtual   
2  01/10/2018 2:00           18.0 -37.82038  144.95746  Sea Life - Virtual   
3  01/10/2018 3:00            9.0 -37.82038  144.95746  Sea Life - Virtual   
4  01/10/2018 4:00           36.0 -37.82038  144.95746  Sea Life - Virtual   

   sensor_id  
0        108  
1        108  
2        108  
3        108  
4        108

### Append Existing DF

In [1851]:
## Change sensor

df3_new = df3_new.append(sensor_i, sort=False).reset_index(drop=True)

In [1852]:
## Change sensor

df2 = df2.append(sensor_i.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor J - Immigration Museum </font>

In [1853]:
sensor10 = pd.DataFrame([{'Latitude':-37.81934,'Longitude':144.96075}])
sensor10

Latitude  Longitude
0 -37.81934  144.96075

### Nearest sensors

In [1854]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor10, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor10, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1855]:
# Nearest 3 sensors
df4 = df4.sort_values('distance')
df4 = df4.reset_index(drop=True)
df4.head(5)

sensor_id   latitude   longitude  distance
0       14.0 -37.820112  144.962919  0.209335
1      101.0 -37.817771  144.957951  0.301768
2      108.0 -37.820380  144.957460  0.311827
3       36.0 -37.816525  144.961211  0.315037
4       35.0 -37.820178  144.965089  0.393186

In [1856]:
a,b,c = 0,1,3
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1857]:
## Change sensor

sensor_j = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

sensor_j.head(5)

date_time  hourly_counts
0  01/10/2018 0:00           59.0
1  01/10/2018 1:00           19.0
2  01/10/2018 2:00           14.0
3  01/10/2018 3:00           13.0
4  01/10/2018 4:00           22.0

In [1858]:
## Change sensor

sensor_j['latitude']= sensor10['Latitude'][0]
sensor_j['longitude']= sensor10['Longitude'][0]
sensor_j['sensor_name']= 'Immigration Museum - Virtual'
sensor_j['sensor_id']= 109
sensor_j.tail(10)

date_time  hourly_counts  latitude  longitude  \
4358  31/03/2019 14:00          516.0 -37.81934  144.96075   
4359  31/03/2019 15:00          440.0 -37.81934  144.96075   
4360  31/03/2019 16:00          470.0 -37.81934  144.96075   
4361  31/03/2019 17:00          425.0 -37.81934  144.96075   
4362  31/03/2019 18:00          446.0 -37.81934  144.96075   
4363  31/03/2019 19:00          334.0 -37.81934  144.96075   
4364  31/03/2019 20:00          319.0 -37.81934  144.96075   
4365  31/03/2019 21:00          238.0 -37.81934  144.96075   
4366  31/03/2019 22:00          176.0 -37.81934  144.96075   
4367  31/03/2019 23:00          103.0 -37.81934  144.96075   

                       sensor_name  sensor_id  
4358  Immigration Museum - Virtual        109  
4359  Immigration Museum - Virtual        109  
4360  Immigration Museum - Virtual        109  
4361  Immigration Museum - Virtual        109  
4362  Immigration Museum - Virtual        109  
4363  Immigration Museum - Virtual        109  
4364  Immigration Museum - Virtual        109  
4365  Immigration Museum - Virtual        109  
4366  Immigration Museum - Virtual        109  
4367  Immigration Museum - Virtual        109

### Append Existing DF

In [1859]:
## Change sensor

df3_new = df3_new.append(sensor_j, sort=False).reset_index(drop=True)

In [1860]:
## Change sensor

df2 = df2.append(sensor_j.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

### <font color='blue'>Sensor K - RMIT </font>

In [1861]:
sensor11 = pd.DataFrame([{'Latitude':-37.80892,'Longitude':144.96634}])
sensor11

Latitude  Longitude
0 -37.80892  144.96634

### Nearest sensors

In [1862]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor11, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor11, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1863]:
# Nearest 3 sensors
df4 = df4.sort_values('distance')
df4 = df4.reset_index(drop=True)
df4.head(3)

sensor_id   latitude   longitude  distance
0      105.0 -37.808234  144.968757  0.226051
1       15.0 -37.810644  144.964471  0.252358
2       30.0 -37.811218  144.966568  0.255906

In [1864]:
a,b,c = 0,1,2
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1865]:
## Change sensor
sensor_k = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

sensor_k.head(5)

date_time  hourly_counts
0  01/10/2018 0:00           96.0
1  01/10/2018 1:00           66.0
2  01/10/2018 2:00           50.0
3  01/10/2018 3:00           31.0
4  01/10/2018 4:00          280.0

In [1866]:
## Change sensor

sensor_k['latitude']= sensor11['Latitude'][0]
sensor_k['longitude']= sensor11['Longitude'][0]
sensor_k['sensor_name']= 'RMIT - Virtual'
sensor_k['sensor_id']= 110
sensor_k.tail(10)

date_time  hourly_counts  latitude  longitude     sensor_name  \
4358  31/03/2019 14:00         1020.0 -37.80892  144.96634  RMIT - Virtual   
4359  31/03/2019 15:00         1025.0 -37.80892  144.96634  RMIT - Virtual   
4360  31/03/2019 16:00          934.0 -37.80892  144.96634  RMIT - Virtual   
4361  31/03/2019 17:00         1103.0 -37.80892  144.96634  RMIT - Virtual   
4362  31/03/2019 18:00         1050.0 -37.80892  144.96634  RMIT - Virtual   
4363  31/03/2019 19:00         1000.0 -37.80892  144.96634  RMIT - Virtual   
4364  31/03/2019 20:00          815.0 -37.80892  144.96634  RMIT - Virtual   
4365  31/03/2019 21:00          592.0 -37.80892  144.96634  RMIT - Virtual   
4366  31/03/2019 22:00          394.0 -37.80892  144.96634  RMIT - Virtual   
4367  31/03/2019 23:00          233.0 -37.80892  144.96634  RMIT - Virtual   

      sensor_id  
4358        110  
4359        110  
4360        110  
4361        110  
4362        110  
4363        110  
4364        110  
4365        110  
4366        110  
4367        110

### Append Existing DF

In [1867]:
## Change sensor

df3_new = df3_new.append(sensor_k, sort=False).reset_index(drop=True)

In [1868]:
## Change sensor

df2 = df2.append(sensor_k.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

## Sensor - 59

In [1869]:
sensor12 = pd.DataFrame([{'Latitude':-37.80825648,'Longitude':144.9630486}])
sensor12

Latitude   Longitude
0 -37.808256  144.963049

### Nearest sensors

In [1870]:
# Find Sensor information and Distance

## Change sensor
df4 = pd.concat([pd.DataFrame(within(sensor12, df2, 1, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(sensor12, df2, 1, 0)).reset_index(drop=True)], axis=1)
df4.rename(columns={0:'distance'}, inplace=True)

In [1871]:
# Nearest 3 sensors
df4 = df4.sort_values('distance')
df4 = df4.reset_index(drop=True)
df4.head(5)

sensor_id   latitude   longitude  distance
0       59.0 -37.808256  144.963049  0.000000
1       26.0 -37.808695  144.960494  0.230174
2       15.0 -37.810644  144.964471  0.293098
3      110.0 -37.808920  144.966340  0.299052
4       49.0 -37.807301  144.959561  0.324962

In [1872]:
a,b,c = 1,2,3
df_nearest = df3_new[(df3_new['sensor_id'] == df4['sensor_id'][a]) |
                 (df3_new['sensor_id'] == df4['sensor_id'][b]) |
                 ((df3_new['sensor_id'] == df4['sensor_id'][c]))]

divider = ((1/df4['distance'][a]) + (1/df4['distance'][b]) + (1/df4['distance'][c]))

d1 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][a]]['hourly_counts']*((1/df4['distance'][a])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][b]]['hourly_counts']*((1/df4['distance'][b])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][c]]['hourly_counts']*((1/df4['distance'][c])/divider)

In [1873]:
## Change sensor
sensor_l = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df4['sensor_id'][1]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

sensor_l.head(5)

date_time  hourly_counts
0  01/10/2018 0:00           95.0
1  01/10/2018 1:00           59.0
2  01/10/2018 2:00           40.0
3  01/10/2018 3:00           31.0
4  01/10/2018 4:00          322.0

In [1874]:
## Change sensor

sensor_l['latitude']= sensor11['Latitude'][0]
sensor_l['longitude']= sensor11['Longitude'][0]
sensor_l['sensor_name']= 'Sensor 59'
sensor_l['sensor_id']= 59
sensor_l.tail(10)

date_time  hourly_counts  latitude  longitude sensor_name  \
4358  31/03/2019 14:00         1480.0 -37.80892  144.96634   Sensor 59   
4359  31/03/2019 15:00         1332.0 -37.80892  144.96634   Sensor 59   
4360  31/03/2019 16:00         1092.0 -37.80892  144.96634   Sensor 59   
4361  31/03/2019 17:00         1116.0 -37.80892  144.96634   Sensor 59   
4362  31/03/2019 18:00         1018.0 -37.80892  144.96634   Sensor 59   
4363  31/03/2019 19:00          976.0 -37.80892  144.96634   Sensor 59   
4364  31/03/2019 20:00          797.0 -37.80892  144.96634   Sensor 59   
4365  31/03/2019 21:00          585.0 -37.80892  144.96634   Sensor 59   
4366  31/03/2019 22:00          364.0 -37.80892  144.96634   Sensor 59   
4367  31/03/2019 23:00          232.0 -37.80892  144.96634   Sensor 59   

      sensor_id  
4358         59  
4359         59  
4360         59  
4361         59  
4362         59  
4363         59  
4364         59  
4365         59  
4366         59  
4367         59

### Append Existing DF

In [1913]:
## Change sensor

df3_new = df3_new.append(sensor_l, sort=False).reset_index(drop=True)

In [1914]:
## Change sensor

df2 = df2.append(sensor_k.iloc[0,[2,3,5]], sort=False).reset_index(drop=True)

## Save

In [1915]:
df2.to_csv('df2.csv', index=False)

In [1945]:
df3_new.to_csv('pedestrian_temp.csv', index=False)

# <font color='red'> For Each Gallery </font>

In [1891]:
from datetime import date
import calendar# Find Sensor information and Distance

In [1917]:
df = pd.read_csv("pedestrian_temp.csv")
gallery = pd.read_csv("../data/Gallery.csv")
df2 = pd.read_csv('df2.csv')

In [1918]:
df.head()

date_time  sensor_id     sensor_name  hourly_counts   latitude  \
0  01/10/2018 0:00          5  Princes Bridge          123.0 -37.818742   
1  01/10/2018 1:00          5  Princes Bridge           46.0 -37.818742   
2  01/10/2018 2:00          5  Princes Bridge           25.0 -37.818742   
3  01/10/2018 3:00          5  Princes Bridge           48.0 -37.818742   
4  01/10/2018 4:00          5  Princes Bridge           50.0 -37.818742   

    longitude  
0  144.967877  
1  144.967877  
2  144.967877  
3  144.967877  
4  144.967877

In [1919]:
# Find weekday
df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%m/%Y %H:%M')

In [1920]:
my_day_list = []
for i in range(len(df)):
    my_date = df['date_time'][i]
    my_day = calendar.day_name[my_date.weekday()]
    my_day_list.append(my_day)

In [1921]:
# Append Column
df['Day'] = my_day_list

In [1922]:
df.head(2)

date_time  sensor_id     sensor_name  hourly_counts   latitude  \
0 2018-10-01 00:00:00          5  Princes Bridge          123.0 -37.818742   
1 2018-10-01 01:00:00          5  Princes Bridge           46.0 -37.818742   

    longitude     Day  
0  144.967877  Monday  
1  144.967877  Monday

In [1923]:
day_list = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday', 'Saturday']

In [1943]:
df=df.drop_duplicates(keep='first')

In [1925]:
def get_ped(gallery, pedestrian, i):

    gallery_sensor = pd.DataFrame([{'Latitude':gallery['Latitude'][i],'Longitude': gallery['Longitude'][i]}])

    # Find Nearest 3 sensors
    df_temp = pd.concat([pd.DataFrame(within(gallery_sensor, df2, 1.5, 0)).reset_index(drop=True),
               pd.DataFrame(within_d(gallery_sensor, df2, 1.5, 0)).reset_index(drop=True)], axis=1)
    df_temp.rename(columns={0:'distance'}, inplace=True)

    # Get Nearest 3 sensors
    df_temp = df_temp.sort_values('distance').iloc[0:3,:]
    df_temp = df_temp.reset_index(drop=True)

    # DF Nearest
    df_nearest = pedestrian[(pedestrian['sensor_id'] == df_temp['sensor_id'][0]) |
                 (pedestrian['sensor_id'] == df_temp['sensor_id'][1]) |
                 ((pedestrian['sensor_id'] == df_temp['sensor_id'][2]))]

#     d1 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][0]]['hourly_counts']*(df_temp['distance'][0])/((df_temp['distance'][0])/df_temp['distance'].sum())
#     d2 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][1]]['hourly_counts']*(df_temp['distance'][1])/((df_temp['distance'][1])/df_temp['distance'].sum())
#     d3 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][2]]['hourly_counts']*(df_temp['distance'][2])/((df_temp['distance'][2])/df_temp['distance'].sum())
    divider = ((1/df_temp['distance'][0]) + (1/df_temp['distance'][1]) + (1/df_temp['distance'][2]))

    d1 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][0]]['hourly_counts']*((1/df_temp['distance'][0])/divider)
    d2 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][1]]['hourly_counts']*((1/df_temp['distance'][1])/divider)
    d3 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][2]]['hourly_counts']*((1/df_temp['distance'][2])/divider)

    sensor_interest = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

    ## Change sensor
    sensor_interest['latitude']= gallery_sensor['Latitude'][0]
    sensor_interest['longitude']= gallery_sensor['Longitude'][0]
    sensor_interest['sensor_name']= gallery['Gallery Name'][i]

    my_day_list = []

    sensor_interest['date_time'] = pd.to_datetime(sensor_interest['date_time'])

    for k in range(len(sensor_interest)):
        my_date = sensor_interest['date_time'][k]
        my_day = calendar.day_name[my_date.weekday()]
        my_day_list.append(my_day)

    sensor_interest['Day'] = my_day_list
    

    return sensor_interest

In [6]:
def for_eachday(df):

    each = []
    day_list = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday', 'Saturday']
    
    # Change time to int
    time = df['date_time'].apply(lambda x: int(x.strftime('%H')))
    df['time'] = time
        
    # Loop over days
    for i in range (7):
        # 10AM - 5PM
        dday = round(df[(df['Day']==day_list[i]) & (df['time'] < 18) & (df['time'] > 9) ]['hourly_counts'].mean())
        each.append(dday)
        
    return each

In [1931]:
# Find average for each gallery
average_list = []

for n in range(0,len(gallery)):
    average = for_eachday(get_ped(gallery, df, i=n))
    average_list.append(average)

In [1930]:
# Concat
past = pd.concat([gallery, pd.DataFrame(average_list, columns=day_list)], axis=1)
past.to_csv('past.csv')
past

Gallery Name   Latitude   Longitude  \
0                        Old Melbourne Gaol Crime -37.807764  144.965464   
1                           Victorian Arts Centre -37.821995  144.968837   
2            The Ian Potter Centre: NGV Australia -37.817483  144.969899   
3                     Thoroughbred Racing Gallery -37.818866  144.983970   
4                      Fox Classic Car Collection -37.821374  144.948497   
5                       Koorie Heritage Trust Inc -37.813385  144.954028   
6   Australian Centre For The Moving Image (ACMI) -37.817611  144.969070   
7                               Blindside Gallery -37.816711  144.966812   
8                           Flinders Lane Gallery -37.815715  144.970033   
9                               Mailbox Art Space -37.815758  144.969861   
10                                   RMIT Gallery -37.812587  144.955458   
11                                      Neon Parc -37.813892  144.964020   
12                                  Outre Gallery -37.812816  144.962224   
13                          Anna Schwartz Gallery -37.816048  144.968890   
14                                ARC ONE Gallery -37.814730  144.973080   
15                                 Gallery Funaki -37.811609  144.971399   
16                         Lesley Kehoe Galleries -37.814922  144.970520   
17                                  SpACE@Collins -37.815775  144.964823   
18                              Tolarno Galleries -37.813133  144.971330   
19                             Immigration Museum -37.818374  144.959207   
20                   National Gallery of Victoria -37.822594  144.968928   
21                                 The Dax Centre -37.798269  144.958551   
22                        Fortyfivedownstairs Inc -37.816956  144.970885   
23                        Pieces of Eight Gallery -37.813492  144.967384   
24                     Magnet Galleries Melbourne -37.816581  144.952387   

    Sunday  Monday  Tuesday  Wednesday  Thursday  Friday  Saturday  
0      844     874      879        929       934     967       889  
1     1993    1638     1411       1720      1542    1590      1900  
2     1100    1169     1006       1154      1082    1147      1071  
3      187     480      483        508       524     528       206  
4      172     691      709        739       742     711       189  
5      777    1227     1199       1288      1343    1350       868  
6     1260    1266     1078       1243      1167    1231      1227  
7     1869    2324     2242       2441      2417    2436      2116  
8      315     821      827        877       915     905       375  
9      361     862      863        918       954     946       427  
10     796     959      940        998      1036    1045       874  
11    1942    2179     1975       2205      2179    2305      2215  
12     715     898      864        929       923     924       808  
13     704    1175     1146       1231      1255    1263       823  
14     192     504      507        534       551     554       213  
15     176     594      620        663       683     683       233  
16     174     668      685        721       763     748       214  
17    1555    2017     1897       2110      2082    2101      1791  
18     189     734      752        792       830     810       230  
19     444    1009     1042       1119      1118    1118       523  
20    2069    1769     1564       1822      1668    1733      1971  
21     147     417      399        425       405     391       156  
22     584     820      730        826       811     855       590  
23    1845    2050     1917       2070      2047    2145      2051  
24     834    1735     1718       1833      1941    1933       951

In [22]:
# Change time to int
# Find weekday
df['date_time'] = pd.to_datetime(df['date_time'], format='%d/%m/%Y %H:%M')
time = df['date_time'].apply(lambda x: int(x.strftime('%H')))
df['time'] = time

In [ ]:
# Loop over days
for i in range (7):
    # 10AM - 5PM
    dday = round(df[(df['Day']==day_list[i]) & (df['time'] < 18) & (df['time'] > 9) ]['hourly_counts'].mean())
    each.append(dday)

In [26]:
df[(df['time'] < 18) & (df['time'] > 9) ].to_csv('pedestrian.csv', index=False)

## Any Location 

In [3]:
click = [144.96542758565607,-37.809645236233656]

In [26]:
from datetime import date
import calendar# Find Sensor information and Distance
import pandas as pd
import geopy
from geopy import distance


df = pd.read_csv("../static/data/pedestrian_temp.csv")
df2 = pd.read_csv('../static/data/sensors.csv')




def within(df, target, radius, g):
    temp = []
    dist = [(geopy.distance.distance((df["Latitude"][g], df["Longitude"][g]),
                                    (target["latitude"][x], target["longitude"][x])).km) for x in range(len(target))]
    for x in range(len(target)):
        if dist[x] < radius:
            temp.append(target.iloc[x])
        else:
            pass

    return temp

def within_d(df, target, radius, g):
    distances = []
    dist = [(geopy.distance.distance((df["Latitude"][g], df["Longitude"][g]),
                                    (target["latitude"][x], target["longitude"][x])).km) for x in range(len(target))]
    for x in range(len(target)):
        if dist[x] < radius:
            distances.append(dist[x])
        else:
            pass

    return distances

# def within(df, target, radius, g):
    
#     temp = []
    
#     for x in range(len(target)):
#         # Use try-except for error handling
#         try:
#             # Find coordinate1,2 calculate distance in km
#             coords_1 = (df["Latitude"][g], df["Longitude"][g])
#             coords_2 = (target["latitude"][x], target["longitude"][x])
#             dist = geopy.distance.distance(coords_1, coords_2).km

#             if dist <= radius:
#                 temp.append(target.iloc[x])
#         except:
#             pass
#     return temp

# def within_d(df, target, radius, g):
    
#     distances = []
    
#     for x in range(len(target)):
#         # Use try-except for error handling
#         try:
#             # Find coordinate1,2 calculate distance in km
#             coords_1 = (df["Latitude"][g], df["Longitude"][g])
#             coords_2 = (target["latitude"][x], target["longitude"][x])
#             dist = geopy.distance.distance(coords_1, coords_2).km

#             if dist <= radius:
#                 distances.append(dist)
#         except:
#             pass
#     return distances


def get_ped_any(click, pedestrian, time=[10,17]):

    click_sensor = pd.DataFrame([{'Latitude':click[1],'Longitude': click[0]}])

    # Find Nearest 3 sensors
    df_temp = pd.concat([pd.DataFrame(within(click_sensor, df2, 1.5, 0)).reset_index(drop=True),
               pd.DataFrame(within_d(click_sensor, df2, 1.5, 0)).reset_index(drop=True)], axis=1)
    df_temp.rename(columns={0:'distance'}, inplace=True)

    # Get Nearest 3 sensors
    df_temp = df_temp.sort_values('distance').iloc[0:3,:]
    df_temp = df_temp.reset_index(drop=True)

    # DF Nearest
    df_nearest = pedestrian[(pedestrian['sensor_id'] == df_temp['sensor_id'][0]) |
                 (pedestrian['sensor_id'] == df_temp['sensor_id'][1]) |
                 ((pedestrian['sensor_id'] == df_temp['sensor_id'][2]))]

    divider = ((1/df_temp['distance'][0]) + (1/df_temp['distance'][1]) + (1/df_temp['distance'][2]))

    d1 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][0]]['hourly_counts']*((1/df_temp['distance'][0])/divider)
    d2 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][1]]['hourly_counts']*((1/df_temp['distance'][1])/divider)
    d3 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][2]]['hourly_counts']*((1/df_temp['distance'][2])/divider)
    
    ## Change sensor
    sensor_interest = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][0]]['date_time'].reset_index(drop=True)),
           pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

    ## Change sensor
    sensor_interest['latitude']= click_sensor['Latitude'][0]
    sensor_interest['longitude']= click_sensor['Longitude'][0]

    my_day_list = []

    sensor_interest['date_time'] = pd.to_datetime(sensor_interest['date_time'])

    for i in range(len(sensor_interest)):
        my_date = sensor_interest['date_time'][i]
        my_day = calendar.day_name[my_date.weekday()]
        my_day_list.append(my_day)

    sensor_interest['Day'] = my_day_list
    

    return sensor_interest



In [27]:
%timeit get_ped_any(click, pedestrian, time=[10,17])

471 ms ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# ori 542 ms ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# _d  467 ms ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# _n  457 ms ± 11.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [5]:
pedestrian = df

click_sensor = pd.DataFrame([{'Latitude':click[1],'Longitude': click[0]}])

# Find Nearest 3 sensors
df_temp = pd.concat([pd.DataFrame(within(click_sensor, df2, 1.5, 0)).reset_index(drop=True),
           pd.DataFrame(within_d(click_sensor, df2, 1.5, 0)).reset_index(drop=True)], axis=1)
df_temp.rename(columns={0:'distance'}, inplace=True)

# Get Nearest 3 sensors
df_temp = df_temp.sort_values('distance').iloc[0:3,:]
df_temp = df_temp.reset_index(drop=True)

# DF Nearest
df_nearest = pedestrian[(pedestrian['sensor_id'] == df_temp['sensor_id'][0]) |
             (pedestrian['sensor_id'] == df_temp['sensor_id'][1]) |
             ((pedestrian['sensor_id'] == df_temp['sensor_id'][2]))]

divider = ((1/df_temp['distance'][0]) + (1/df_temp['distance'][1]) + (1/df_temp['distance'][2]))

d1 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][0]]['hourly_counts']*((1/df_temp['distance'][0])/divider)
d2 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][1]]['hourly_counts']*((1/df_temp['distance'][1])/divider)
d3 = df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][2]]['hourly_counts']*((1/df_temp['distance'][2])/divider)

## Change sensor
sensor_interest = pd.concat([pd.DataFrame(df_nearest[df_nearest['sensor_id'] == df_temp['sensor_id'][0]]['date_time'].reset_index(drop=True)),
       pd.DataFrame({'hourly_counts':(d1.values+d2.values+d3.values).round()})], axis=1)

## Change sensor
sensor_interest['latitude']= click_sensor['Latitude'][0]
sensor_interest['longitude']= click_sensor['Longitude'][0]

my_day_list = []

sensor_interest['date_time'] = pd.to_datetime(sensor_interest['date_time'])

for i in range(len(sensor_interest)):
    my_date = sensor_interest['date_time'][i]
    my_day = calendar.day_name[my_date.weekday()]
    my_day_list.append(my_day)

sensor_interest['Day'] = my_day_list

sensor_interest

Wall time: 0 ns


date_time  hourly_counts   latitude   longitude        Day
0    2018-01-10 00:00:00          117.0 -37.809645  144.965428  Wednesday
1    2018-01-10 01:00:00           77.0 -37.809645  144.965428  Wednesday
2    2018-01-10 02:00:00           60.0 -37.809645  144.965428  Wednesday
3    2018-01-10 03:00:00           35.0 -37.809645  144.965428  Wednesday
4    2018-01-10 04:00:00          230.0 -37.809645  144.965428  Wednesday
5    2018-01-10 05:00:00           31.0 -37.809645  144.965428  Wednesday
6    2018-01-10 06:00:00           58.0 -37.809645  144.965428  Wednesday
7    2018-01-10 07:00:00          215.0 -37.809645  144.965428  Wednesday
8    2018-01-10 08:00:00          516.0 -37.809645  144.965428  Wednesday
9    2018-01-10 09:00:00          494.0 -37.809645  144.965428  Wednesday
10   2018-01-10 10:00:00          616.0 -37.809645  144.965428  Wednesday
11   2018-01-10 11:00:00          934.0 -37.809645  144.965428  Wednesday
12   2018-01-10 12:00:00         1372.0 -37.809645  144.965428  Wednesday
13   2018-01-10 13:00:00         1346.0 -37.809645  144.965428  Wednesday
14   2018-01-10 14:00:00         1086.0 -37.809645  144.965428  Wednesday
15   2018-01-10 15:00:00         1074.0 -37.809645  144.965428  Wednesday
16   2018-01-10 16:00:00         1095.0 -37.809645  144.965428  Wednesday
17   2018-01-10 17:00:00         1121.0 -37.809645  144.965428  Wednesday
18   2018-01-10 18:00:00         1008.0 -37.809645  144.965428  Wednesday
19   2018-01-10 19:00:00          840.0 -37.809645  144.965428  Wednesday
20   2018-01-10 20:00:00          702.0 -37.809645  144.965428  Wednesday
21   2018-01-10 21:00:00          501.0 -37.809645  144.965428  Wednesday
22   2018-01-10 22:00:00          326.0 -37.809645  144.965428  Wednesday
23   2018-01-10 23:00:00          178.0 -37.809645  144.965428  Wednesday
24   2018-02-10 00:00:00          115.0 -37.809645  144.965428   Saturday
25   2018-02-10 01:00:00           87.0 -37.809645  144.965428   Saturday
26   2018-02-10 02:00:00           35.0 -37.809645  144.965428   Saturday
27   2018-02-10 03:00:00           28.0 -37.809645  144.965428   Saturday
28   2018-02-10 04:00:00           23.0 -37.809645  144.965428   Saturday
29   2018-02-10 05:00:00           49.0 -37.809645  144.965428   Saturday
...                  ...            ...        ...         ...        ...
4338 2019-03-30 18:00:00         1217.0 -37.809645  144.965428   Saturday
4339 2019-03-30 19:00:00         1300.0 -37.809645  144.965428   Saturday
4340 2019-03-30 20:00:00         1218.0 -37.809645  144.965428   Saturday
4341 2019-03-30 21:00:00         1071.0 -37.809645  144.965428   Saturday
4342 2019-03-30 22:00:00          907.0 -37.809645  144.965428   Saturday
4343 2019-03-30 23:00:00          767.0 -37.809645  144.965428   Saturday
4344 2019-03-31 00:00:00          510.0 -37.809645  144.965428     Sunday
4345 2019-03-31 01:00:00          340.0 -37.809645  144.965428     Sunday
4346 2019-03-31 02:00:00          250.0 -37.809645  144.965428     Sunday
4347 2019-03-31 03:00:00          195.0 -37.809645  144.965428     Sunday
4348 2019-03-31 04:00:00          141.0 -37.809645  144.965428     Sunday
4349 2019-03-31 05:00:00          112.0 -37.809645  144.965428     Sunday
4350 2019-03-31 06:00:00           66.0 -37.809645  144.965428     Sunday
4351 2019-03-31 07:00:00           80.0 -37.809645  144.965428     Sunday
4352 2019-03-31 08:00:00          132.0 -37.809645  144.965428     Sunday
4353 2019-03-31 09:00:00          269.0 -37.809645  144.965428     Sunday
4354 2019-03-31 10:00:00          458.0 -37.809645  144.965428     Sunday
4355 2019-03-31 11:00:00          685.0 -37.809645  144.965428     Sunday
4356 2019-03-31 12:00:00          961.0 -37.809645  144.965428     Sunday
4357 2019-03-31 13:00:00         1138.0 -37.809645  144.965428     Sunday
4358 2019-03-31 14:00:00         1084.0 -37.809645  144.965428     Sunday
4359 2019-03-31 15:00:00         1079.0 -37.809645  144.965428     Sunday
4360 2019-03-31 16: